In [387]:
#Install and Authenticate with ibm_watson tone analyser

from ibm_watson import ToneAnalyzerV3
from ibm_cloud_sdk_core.authenticators import IAMAuthenticator

authenticator = IAMAuthenticator('Z-6GpPGH9xLcC-0maf37aOxnBGbqV-1jHEYoVGKYo7Tx')
tone_analyzer = ToneAnalyzerV3(
    version='2020-07-26',
    authenticator=authenticator
)

tone_analyzer.set_service_url('https://api.us-south.tone-analyzer.watson.cloud.ibm.com/instances/c470c928-790d-4c42-89a2-b6c145810c99')

In [ ]:
#Disable SSL if I need to
#tone_analyzer.set_disable_ssl_verification(True)

In [388]:
import json
from ibm_watson import ApiException


In [389]:
def get_emotion(questiontext):
    tone_input = questiontext
    content_typeParam ='text/plain;charset=utf-8'
    sentencesParam = True #whether sentences should be analysed in addition to document.
    tonesParam = None #deprecated
    content_languageParam = 'en'  #english
    accept_languageParam = 'en' #english for response

    from ibm_watson import ApiException
    try:
        # Invoke a Tone Analyzer method
        tone_analysis = tone_analyzer.tone(tone_input, content_type=content_typeParam, sentences=sentencesParam, tones=tonesParam, content_language=content_languageParam, accept_language=accept_languageParam)
    except ApiException as ex:
        print("Method failed with status code " + str(ex.code) + ": " + ex.message)

    #Tone analysis methods: get_results(), get_headers(), get_status_code()
    #print(tone_analysis)
    return tone_analysis


In [337]:
#Nicely formated timespan from Jeff Heaton
def hms_string(sec_elapsed):
    h = int(sec_elapsed / (60*60))
    m = int((sec_elapsed % (60*60)) / 60)
    s = sec_elapsed % 60
    return "{}:{:>02}:{:>05.2f}".format(h, m, s)

In [342]:
#Read in the visionAPIClassificationFile - the important columns are "ID" and "Question Text"
import pandas as pd
visionapi_df = pd.DataFrame
filename = 'C:\\Users\\umg\\Development\\SEmotion\\03_Journal\\Data\VisionAPIQuestions_source.csv'
visionapi_df = pd.read_csv(filename, sep=',', )

In [333]:
testtext = visionapi_df.loc[1, 'Question Text']
print(testtext)

"I have problem with access to camera first time. I installed my app. Next go to scan to QR Code. I use to scan QR Code google vision. App show dialog, which show about the permissions, next click "Allow",but camera doesn't open. But I go back activity and go to activity which scan QR Code, camera open.my AdnroidManifest.xmlmy class"


In [334]:
visionapi_df.head()

,ID,FearID,fear,JoyID,joy,LoveID,love,SadID,Sadness,SurpriseID,surprise,AngerID,anger,NoEmotionID,none,NoOfEmotions,Include,Question Text
0,45033467,45033467,fear,45033467,NoJoy,45033467,love,45033467,notsadness,45033467,notsurprise,45033467,NotAnger,45033467,NaN,2.0,Yes,"""I want to build a cloud based solution in whi..."
1,45546462,45546462,fear,45546462,NoJoy,45546462,notlove,45546462,notsadness,45546462,notsurprise,45546462,NotAnger,45546462,NaN,1.0,YES,"""I have problem with access to camera first ti..."
2,49619897,49619897,notfear,49619897,NoJoy,49619897,notlove,49619897,notsadness,49619897,surprise,49619897,Anger,49619897,NaN,2.0,Yes,"""Using Airflow I want to get the result of an ..."
3,55388663,55388663,notfear,55388663,NoJoy,55388663,notlove,55388663,notsadness,55388663,notsurprise,55388663,NotAnger,55388663,NoEmotion,0.0,Yes,"""I try to use Google Cloud Vision API to detec..."
4,52833231,52833231,notfear,52833231,NoJoy,52833231,notlove,52833231,notsadness,52833231,notsurprise,52833231,NotAnger,52833231,NoEmotion,0.0,Yes,"""How can i write a test case in Junit for amaz..."


In [390]:
#Process the Vision API dataframe and collect the data into a list [Question no, tone_analyser object]
import time
totalCount = 0
start_time = time.time()
iteration_df = visionapi_df.copy()  # take a copy of the dataframe

doctone_df = pd.DataFrame(columns = ['QuestionID', 'Question Text', 'JoyToneID', 'JoyScore', 'SadnessToneID','SadnessScore','FearToneID', 'FearScore', 'AngerToneID', 'AngerScore', 'AnalyticalToneID','AnalyticalScore','ConfidentToneID', 'ConfidentScore','TentativeToneID', 'TentativeScore', 'NoEmotion'])
document_emotion_pd = pd.DataFrame()

sentencetone_df = pd.DataFrame(columns = ['QuestionID', 'Question Text', 'SentenceID', 'Sentence', 'ToneSummary','JoyToneID', 'JoyScore', 'SadnessToneID','SadnessScore','FearToneID', 'FearScore', 'AngerToneID', 'AngerScore', 'AnalyticalToneID','AnalyticalScore','ConfidentToneID', 'ConfidentScore','TentativeToneID', 'TentativeScore', 'NoEmotion'])
sentence_summary_emotion_df = pd.DataFrame()
sentence_detail_emotion_df = pd.DataFrame()
sentence_detail_emotions_df_init = sentence_detail_emotion_df.copy()


for index, row in iteration_df.iterrows():
    totalCount+=1
    if index >=0 :
        #result = get_emotion(row['Question Text']) 
        ta = get_emotion(row.loc['Question Text']) 
        result = ta.get_result()  #ta = tone analysis object
        #print("result ", result)
        #print("question", row.loc['Question Text'])
    
        JoyToneID = False
        JoyScore = 0
        SadnessToneID = False
        SadnessScore = 0
        FearToneID = False
        FearScore = 0
        AngerToneID = False
        AngerScore = 0 
        AnalyticalToneID = False
        AnalyticalScore = 0
        ConfidentToneID = False
        ConfidentScore = 0
        TentativeToneID = False
        TentativeScore = 0
        NoEmotion = True
        
        # Document related process
        document_emotion_pd = pd.json_normalize(result['document_tone'], 'tones')
        #print("first document emotion pd:  ", document_emotion_pd)
    
        if (document_emotion_pd.empty == False): 
            document_emotion_pd = document_emotion_pd.set_index(document_emotion_pd['tone_id'])
            #print("second document emotion pd reindex:  ", document_emotion_pd)
    
            tones = set(pd.Series(document_emotion_pd['tone_id']))
            #print("tones", tones)
    
            #Sadness
            if 'sadness' in tones:
                SadnessToneID = True
                SadnessScore = document_emotion_pd.loc['sadness','score']
    
            #Joy
            if 'joy' in tones:
                JoyToneID = True
                JoyScore = document_emotion_pd.loc['joy','score']
           
            #Fear
            if 'fear' in tones:
                FearToneID = True
                FearScore = document_emotion_pd.loc['fear','score']
        
            #Anger
            if 'anger' in tones:
                AngerToneID = True
                AngerScore = document_emotion_pd.loc['anger','score']
    
            #Analytical
            if 'analytical' in tones:
                AnalyticalToneID = True
                AnalyticalScore = document_emotion_pd.loc['analytical','score']
     
            #Confident
            if 'confident' in tones:
                ConfidentToneID = True
                ConfidentScore = document_emotion_pd.loc['confident','score']
        
            #Tentative
            if 'tentative' in tones:
                TentativeToneID = True
                TentativeScore = document_emotion_pd.loc['tentative','score']
       
            NoEmotion = not (('joy'in tones) or ('sadness' in tones) or ('fear' in tones) or ('anger' in tones))
    
    
        doctone_df = doctone_df.append({'QuestionID' : row['ID'],
                        'Question Text' : row['Question Text'],
                        'JoyToneID' : JoyToneID,
                        'JoyScore': JoyScore,
                        'SadnessToneID': SadnessToneID,
                        'SadnessScore': SadnessScore,
                        'FearToneID': FearToneID,
                        'FearScore': FearScore,
                        'AngerToneID': AngerToneID,
                        'AngerScore': AngerScore,
                        'AnalyticalToneID': AnalyticalToneID,
                        'AnalyticalScore' : AnalyticalScore,              
                        'ConfidentToneID' : ConfidentToneID,
                        'ConfidentScore': ConfidentScore,
                        'TentativeToneID':TentativeToneID,
                        'TentativeScore':TentativeScore,
                        'NoEmotion' : NoEmotion}, ignore_index = True)

    
        # Sentence related process
    
    
        #get all the sentences and details related to the questions
   
        if ('sentences_tone' in result):  
            sentence_summary_emotion_df = pd.json_normalize(result['sentences_tone'])
            sentences_detail_emotions_df = pd.json_normalize(result['sentences_tone'], 'tones',['sentence_id', 'text'], max_level = None, errors='ignore')
            sentences_detail_emotions_df = sentences_detail_emotions_df.set_index(sentences_detail_emotions_df['sentence_id'])
            #print("sentences_detail_emotions_df contents", sentences_detail_emotions_df)
    
            #Process each sentence in the document separately i.e. 1 row per sentence.
            for s_index, s_row in sentence_summary_emotion_df.iterrows():
                    #print("process sentence row", s_row.loc['sentence_id'], s_row.loc['text'])
                    JoyToneID = False
                    JoyScore = 0
                    SadnessToneID = False
                    SadnessScore = 0
                    FearToneID = False
                    FearScore = 0
                    AngerToneID = False
                    AngerScore = 0 
                    AnalyticalToneID = False
                    AnalyticalScore = 0
                    ConfidentToneID = False
                    ConfidentScore = 0
                    TentativeToneID = False
                    TentativeScore = 0
                    NoEmotion = True
        
                    if len(s_row.loc['tones']) > 0: #if there are tones in the sentence: 
                        #print("sentence idex", s_index)
                        s_emotions_df = sentence_detail_emotions_df_init.copy()
                        s_emotions_df = s_emotions_df.append(sentences_detail_emotions_df.loc[s_row.loc['sentence_id']]) #get the detailed emotions rows relating to the sentence_id
                
                        #print("first ", type(s_emotions_pd))
                        s_emotions_df = s_emotions_df.set_index(s_emotions_df['tone_id']) #set emotion tone id as index
                        #print("second ", type(s_emotions_df))
                        tones = set(pd.Series(s_emotions_df['tone_id']))  #get a set of all tones in the emotions
                        #print(tones)
        
                        #now check for individual emotions in the sentence
                        #Sadness
                        if 'sadness' in tones:
                            SadnessToneID = True
                            SadnessScore = s_emotions_df.loc['sadness','score']
    
                        #Joy
                        if 'joy' in tones:
                            JoyToneID = True
                            JoyScore = s_emotions_df.loc['joy','score']
           
                        #Fear
                        if 'fear' in tones:
                            FearToneID = True
                            FearScore = s_emotions_df.loc['fear','score']
        
                        #Anger
                        if 'anger' in tones:
                            AngerToneID = True
                            AngerScore = s_emotions_df.loc['anger','score']
    
                        #Analytical
                        if 'analytical' in tones:
                            AnalyticalToneID = True
                            AnalyticalScore = s_emotions_df.loc['analytical','score']
     
                        #Confident
                        if 'confident' in tones:
                            ConfidentToneID = True
                            ConfidentScore = s_emotions_df.loc['confident','score']
        
                        #Tentative
                        if 'tentative' in tones:
                            TentativeToneID = True
                            TentativeScore = s_emotions_df.loc['tentative','score']
       
                        NoEmotion = not (('joy'in tones) or ('sadness' in tones) or ('fear' in tones) or ('anger' in tones))
    
                    # Contstruct the new row for each sentence
                    sentencetone_df = sentencetone_df.append({'QuestionID' : row['ID'],
                                'QuestionText' : row['Question Text'],
                                'SentenceID':s_row.loc['sentence_id'],
                                'SentenceText':s_row.loc['text'],
                                'ToneSummary':s_row.loc['tones'],
                                'JoyToneID' : JoyToneID,
                                'JoyScore': JoyScore,
                                'SadnessToneID': SadnessToneID,
                                'SadnessScore': SadnessScore,
                                'FearToneID': FearToneID,
                                'FearScore': FearScore,
                                'AngerToneID': AngerToneID,
                                'AngerScore': AngerScore,
                                'AnalyticalToneID': AnalyticalToneID,
                                'AnalyticalScore' : AnalyticalScore,              
                                'ConfidentToneID' : ConfidentToneID,
                                'ConfidentScore': ConfidentScore,
                                'TentativeToneID':TentativeToneID,
                                'TentativeScore':TentativeScore,
                                'NoEmotion' : NoEmotion}, ignore_index = True)
 
        if totalCount > 1 and ((totalCount % 5) == 0):
                 print("We have processed questions {:,}".format(totalCount))
                 #break;
        
time_took = time.time() - start_time
print(f"Total pre-processing runtime: {hms_string(time_took)}")
documentToneAnalyserfn = 'C:\\Users\\umg\\Development\\SEmotion\\03_Journal\\Data\VisionAPIQuestions_DocumentToneAnalyserP2.csv'
sentenceToneAnalyserfn = 'C:\\Users\\umg\\Development\\SEmotion\\03_Journal\\Data\VisionAPIQuestions_SentenceToneAnalyserP2.csv'
doctone_df.to_csv(documentToneAnalyserfn)
sentencetone_df.to_csv(sentenceToneAnalyserfn)


We have processed questions 945
We have processed questions 950
We have processed questions 955
We have processed questions 960
We have processed questions 965
We have processed questions 970
We have processed questions 975
We have processed questions 980
We have processed questions 985
We have processed questions 990
We have processed questions 995
We have processed questions 1,000
We have processed questions 1,005
We have processed questions 1,010
We have processed questions 1,015
We have processed questions 1,020
We have processed questions 1,025
We have processed questions 1,030
We have processed questions 1,035
We have processed questions 1,040
We have processed questions 1,045
We have processed questions 1,050
We have processed questions 1,055
We have processed questions 1,060
We have processed questions 1,065
We have processed questions 1,070
We have processed questions 1,075
We have processed questions 1,080
We have processed questions 1,085
We have processed questions 1,090
We

Working Cells

In [361]:
sentence_summary_emotion_df

,sentence_id,text,tones
0,0,"""I try to use Google Cloud Vision API to detec...",[]
1,1,"After detecting, I get 1 page and 17 Blocks.","[{'score': 0.52908, 'tone_id': 'sadness', 'ton..."
2,2,I am trying to get text in each blocks and sav...,"[{'score': 0.67159, 'tone_id': 'sadness', 'ton..."
3,3,Here is my code:I would like to know is there ...,"[{'score': 0.538448, 'tone_id': 'analytical', ..."
4,4,"Thanks a lot.""","[{'score': 0.808709, 'tone_id': 'joy', 'tone_n..."


In [365]:
 sentences_detail_emotions_df

,score,tone_id,tone_name,sentence_id,text
sentence_id,,,,,
1,0.529080,sadness,Sadness,1,"After detecting, I get 1 page and 17 Blocks."
1,0.620279,analytical,Analytical,1,"After detecting, I get 1 page and 17 Blocks."
2,0.671590,sadness,Sadness,2,I am trying to get text in each blocks and sav...
2,0.638807,analytical,Analytical,2,I am trying to get text in each blocks and sav...
3,0.538448,analytical,Analytical,3,Here is my code:I would like to know is there ...
3,0.775166,tentative,Tentative,3,Here is my code:I would like to know is there ...
4,0.808709,joy,Joy,4,"Thanks a lot."""
4,0.946222,tentative,Tentative,4,"Thanks a lot."""


In [367]:
tones

{'joy', 'tentative'}

In [391]:
result = tone_analysis.get_result()
result

{'document_tone': {'tones': [{'score': 0.629468,
    'tone_id': 'sadness',
    'tone_name': 'Sadness'},
   {'score': 0.603469, 'tone_id': 'analytical', 'tone_name': 'Analytical'}]},
 'sentences_tone': [{'sentence_id': 0,
   'text': '"I have problem with access to camera first time.',
   'tones': [{'score': 0.702427, 'tone_id': 'sadness', 'tone_name': 'Sadness'},
    {'score': 0.724236, 'tone_id': 'analytical', 'tone_name': 'Analytical'}]},
  {'sentence_id': 1, 'text': 'I installed my app.', 'tones': []},
  {'sentence_id': 2, 'text': 'Next go to scan to QR Code.', 'tones': []},
  {'sentence_id': 3,
   'text': 'I use to scan QR Code google vision.',
   'tones': []},
  {'sentence_id': 4,
   'text': 'App show dialog, which show about the permissions, next click "Allow",but camera doesn\'t open.',
   'tones': [{'score': 0.532616,
     'tone_id': 'analytical',
     'tone_name': 'Analytical'}]},
  {'sentence_id': 5,
   'text': 'But I go back activity and go to activity which scan QR Code, cam

In [36]:
header = tone_analysis.get_headers()
print(header)
print(type(header))

{'Content-Type': 'application/json', 'Content-Length': '1079', 'X-Powered-By': 'Servlet/3.1', 'Access-Control-Allow-Origin': '*', 'X-Service-Api-Version': 'null; 2017-09-21', 'X-Service-Build-Number': '2020-07-20T05:01:47', 'Cache-Control': 'no-store', 'Pragma': 'no-cache', 'X-XSS-Protection': '1; mode=block', 'X-Content-Type-Options': 'nosniff', 'Content-Security-Policy': "default-src 'none'", 'Content-Language': 'en-US', 'strict-transport-security': 'max-age=31536000; includeSubDomains;', 'x-global-transaction-id': '26f982c1173138161a0b214a935d2422', 'X-DP-Watson-Tran-ID': '26f982c1173138161a0b214a935d2422', 'X-EdgeConnect-MidMile-RTT': '243', 'X-EdgeConnect-Origin-MEX-Latency': '464', 'Date': 'Sun, 26 Jul 2020 10:00:49 GMT', 'Connection': 'keep-alive'}
<class 'requests.structures.CaseInsensitiveDict'>


In [39]:
status = tone_analysis.get_status_code()
print(status)
print(type(status))

200
<class 'int'>


In [224]:
result

{'document_tone': {'tones': [{'score': 0.629468,
    'tone_id': 'sadness',
    'tone_name': 'Sadness'},
   {'score': 0.603469, 'tone_id': 'analytical', 'tone_name': 'Analytical'}]},
 'sentences_tone': [{'sentence_id': 0,
   'text': '"I have problem with access to camera first time.',
   'tones': [{'score': 0.702427, 'tone_id': 'sadness', 'tone_name': 'Sadness'},
    {'score': 0.724236, 'tone_id': 'analytical', 'tone_name': 'Analytical'}]},
  {'sentence_id': 1, 'text': 'I installed my app.', 'tones': []},
  {'sentence_id': 2, 'text': 'Next go to scan to QR Code.', 'tones': []},
  {'sentence_id': 3,
   'text': 'I use to scan QR Code google vision.',
   'tones': []},
  {'sentence_id': 4,
   'text': 'App show dialog, which show about the permissions, next click "Allow",but camera doesn\'t open.',
   'tones': [{'score': 0.532616,
     'tone_id': 'analytical',
     'tone_name': 'Analytical'}]},
  {'sentence_id': 5,
   'text': 'But I go back activity and go to activity which scan QR Code, cam